In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Sleep Detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Sleep Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TraningConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path:Path
    training_data:Path
    params_epochs:int
    params_batch_size:int
    params_image_size:int
    params_is_augmentation:bool





In [6]:
from src.Sleep_Detection.constants import *
from src.Sleep_Detection.utils.common import read_yaml,create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH):
        self.params=read_yaml(params_filepath)
        self.config=read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])
    def get_training_config(self) ->TraningConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,'dataset/train')
        create_directories([Path(training.root_dir)])

        training_config=TraningConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
        )
        return training_config



In [8]:
import os
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self,config:TraningConfig):
        self.config=config
    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def train_valid_generator(self):
        datagenerator=dict(rescale=1./255,
                           validation_split=0.2)
        dataflow=dict(target_size=self.config.params_image_size[:-1],
                      batch_size=self.config.params_batch_size,
                      interpolation='bilinear')
        validation_datagenrator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator
        )
        self.valid_generator=validation_datagenrator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow
        )
        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=90,
                horizontal_flip=True,
                **datagenerator
            )
        else:
            train_datagenerator=validation_datagenrator
        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow
        )
        
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
            

    def train(self):
        self.step_per_epoch=self.train_generator.samples//self.train_generator.batch_size
        self.validation_epoch=self.valid_generator.samples//self.valid_generator.batch_size
        self.model.fit(self.train_generator,epochs=self.config.params_epochs,
        steps_per_epoch=self.step_per_epoch,
        validation_steps=self.validation_epoch,
        validation_data=self.valid_generator)

        self.save_model(path=self.config.trained_model_path,model=self.model)

In [10]:
try:    
    config=ConfigurationManager()
    trainer_config=config.get_training_config()
    training=Training(config=trainer_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e


[2024-02-17 20:19:06,884: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-17 20:19:06,886: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-17 20:19:06,887: INFO: common: created directory at: artifacts]
[2024-02-17 20:19:06,888: INFO: common: created directory at: artifacts\training]
Found 230 images belonging to 2 classes.
Found 923 images belonging to 2 classes.
Epoch 1/15
28/28 [==============================] - 139s 5s/step - loss: 3.9944 - accuracy: 0.5937 - val_loss: 2.4891 - val_accuracy: 0.5312
Epoch 2/15
28/28 [==============================] - 124s 4s/step - loss: 0.9375 - accuracy: 0.6251 - val_loss: 0.4603 - val_accuracy: 0.7768
Epoch 3/15
28/28 [==============================] - 125s 4s/step - loss: 0.5082 - accuracy: 0.7632 - val_loss: 0.4655 - val_accuracy: 0.7946
Epoch 4/15
28/28 [==============================] - 129s 5s/step - loss: 0.5084 - accuracy: 0.7430 - val_loss: 0.5991 - val_accuracy: 0.6562
Epoch 5/15
28/28 [======

In [ ]:
import pickle
with open(r'C:\Users\\bisht\\OneDrive\Desktop\\Sleep Detection\\artifacts\\training\\model.h5','rb') as f:
    model=f.

AttributeError: '_io.BufferedReader' object has no attribute 'load'